In [1]:
import json
import numpy as np
from sklearn.cluster import KMeans

# Load embeddings from the JSON files
embedding_files = ['embeddings/animals.json', 'embeddings/companies.json', 'embeddings/computer_components.json', 'embeddings/events.json',
                  'embeddings/landmarks.json','embeddings/movies.json','embeddings/people.json',
                  'embeddings/research_fields.json','embeddings/university.json','embeddings/vehicles.json', 'embeddings/gold.json']

all_embeddings = []
file_names = []  # To store the corresponding file names

# Iterate through each file and load its content
for file in embedding_files:
    with open(file, 'r') as f:
        data = json.load(f)  # Load the data from each file
        
        # Add embeddings and file names
        embeddings = np.array([item['embedding'] for item in data])  # Extract 'embedding' key
        all_embeddings.append(embeddings)  # Add the embeddings
        file_names.extend([item['file_name'] for item in data])  # Store file names

# Combine all embeddings into a single array
all_embeddings = np.concatenate(all_embeddings, axis=0)

# Check the shape of the combined embeddings
print("Combined embeddings shape:", all_embeddings.shape)


Combined embeddings shape: (210, 1536)


In [2]:
from sklearn.cluster import KMeans

# Apply k-means clustering with 40 clusters
kmeans40 = KMeans(n_clusters=40, random_state=42)
clusters40 = kmeans40.fit_predict(all_embeddings)

# Check the cluster assignments
print(clusters40)

[29 29 29 29 29 37 37 37 37 37  3  3  3  3  3 36 36 36 36 36 20 20 20 20
 20 27 27 27 27 27  5  5  5  5  5 22 22 22 22 22 31 31 31 31 31 25 25 25
 25 25  6  6  6  6  6 30 30 30 30 30 16 16 16 16 16 10 10 10 10 10 28 28
 28 28 28  0  0  0  0  0 12 12 12 12 12 34 34 34 34 34  4  4  4  4  4 18
 18 18 18 18 24 24 24 24 24  9  9  9  9  9 19 19 19 19 19 13 13 13 13 13
 11 11 11 11 11  1  1  1  1  1  1  1  1  1  1 17 17 17 17 17  8  8  8  8
  8 21 21 21 21 21 39 39 39 39 39 33 33 33 33 33  7  7  7  7  7 32 32 32
 32 32 15 15 15 15 15 35 35 35 35 35  2  2  2  2  2 23 23 23 23 23 26 26
 26 26 26 38 38 38 38 38  2  7  4  3 25  1 14  5 10 13]


In [3]:
# Print the clusters with the corresponding filenames
for cluster_num in range(40):
    cluster_members = [file_names[idx] for idx in range(len(clusters40)) if clusters40[idx] == cluster_num]
    print(f"Cluster {cluster_num}: {cluster_members}")


Cluster 0: ['world_war1.txt', 'world_war2.txt', 'world_war3.txt', 'world_war4.txt', 'world_war5.txt']
Cluster 1: ['havel1.txt', 'havel2.txt', 'havel3.txt', 'havel4.txt', 'havel5.txt', 'klaus1.txt', 'klaus2.txt', 'klaus3.txt', 'klaus4.txt', 'klaus5.txt', 'gold_havel.txt']
Cluster 2: ['airplane1.txt', 'airplane2.txt', 'airplane3.txt', 'airplane4.txt', 'airplane5.txt', 'gold_airplane.txt']
Cluster 3: ['elephant1.txt', 'elephant2.txt', 'elephant3.txt', 'elephant4.txt', 'elephant5.txt', 'gold_elephant.txt']
Cluster 4: ['eiffel_tower1.txt', 'eiffel_tower2.txt', 'eiffel_tower3.txt', 'eiffel_tower4.txt', 'eiffel_tower5.txt', 'gold_eiffel_tower.txt']
Cluster 5: ['microsoft1.txt', 'microsoft2.txt', 'microsoft3.txt', 'microsoft4.txt', 'microsoft5.txt', 'gold_microsoft.txt']
Cluster 6: ['motherboard1.txt', 'motherboard2.txt', 'motherboard3.txt', 'motherboard4.txt', 'motherboard5.txt']
Cluster 7: ['cuni1.txt', 'cuni2.txt', 'cuni3.txt', 'cuni4.txt', 'cuni5.txt', 'gold_cuni.txt']
Cluster 8: ['chemist

In [4]:
import os, re
def extract_entity(fn):
    base = os.path.splitext(fn)[0]
    base = base.replace('gold_', '').replace('_gold', '')
    return re.sub(r'\d+$', '', base)

true_entities = [extract_entity(fn) for fn in file_names]

# H1: define 10 “domain” groups
expected_groups = {
    0: ['havel','klaus','einstein','mandela'],
    1: ['chemistry','economics','mathematics','physics'],
    2: ['amazon','google','microsoft','tencent'],
    3: ['airplane','boat','car','train'],
    4: ['boston_tea','moon_landing','velvet_revolution','world_war'],
    5: ['buckingham_palace','charles_bridge','eiffel_tower','fuji'],
    6: ['cat','dolphin','elephant','giraffe'],
    7: ['cpu','gpu','motherboard','ram'],
    8: ['cuni','mit','oxford','vse'],
    9: ['lord_rings','matrix','pirates','star_wars']
}
entity_to_group = {ent: grp for grp, ents in expected_groups.items() for ent in ents}

# H2: one cluster per entity
unique_entities = sorted(set(true_entities))
entity_to_idx    = {ent: idx for idx, ent in enumerate(unique_entities)}

# 5) Build ground-truth labels
groundtruth10_gold = [entity_to_group[ent]   for ent in true_entities]
groundtruth40_gold = [entity_to_idx[ent]     for ent in true_entities]

invalid = [e for e in true_entities if e not in entity_to_group]
if invalid:
    raise ValueError(f"Unmapped in H1 groups: {set(invalid)}")

print("First 10 filenames:      ", file_names[:10])
print("First 10 extracted ents: ", true_entities[:10])
print("First 10 y_true10 labels:", groundtruth10_gold[:10])
print("First 10 y_true40 labels:", groundtruth40_gold[:10])


First 10 filenames:       ['cat1.txt', 'cat2.txt', 'cat3.txt', 'cat4.txt', 'cat5.txt', 'dolphin1.txt', 'dolphin2.txt', 'dolphin3.txt', 'dolphin4.txt', 'dolphin5.txt']
First 10 extracted ents:  ['cat', 'cat', 'cat', 'cat', 'cat', 'dolphin', 'dolphin', 'dolphin', 'dolphin', 'dolphin']
First 10 y_true10 labels: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
First 10 y_true40 labels: [6, 6, 6, 6, 6, 11, 11, 11, 11, 11]


In [5]:
from sklearn.metrics import rand_score
rand_index = rand_score(groundtruth40_gold, clusters40)

print(f"Rand Index: {rand_index}")

Rand Index: 0.9984051036682615


In [6]:
from sklearn.metrics import adjusted_rand_score

ari40 = adjusted_rand_score(groundtruth40_gold, clusters40)
print(f"Adjusted Rand Index: {ari40:.4f}")

Adjusted Rand Index: 0.9613


In [7]:
# Apply k-means clustering with 10 clusters
kmeans10 = KMeans(n_clusters=10, random_state=42)
clusters10 = kmeans10.fit_predict(all_embeddings)

# Check the cluster assignments
print(clusters10)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 1 1 1 1
 1 0 0 0 0 0 3 3 3 3 3 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 9 3 3 4 1 8 6 5 7]


In [8]:
# Now print the clusters with the corresponding filenames
for cluster_num in range(10):
    cluster_members = [file_names[idx] for idx in range(len(clusters10)) if clusters10[idx] == cluster_num]
    print(f"Cluster {cluster_num}: {cluster_members}")

Cluster 0: ['world_war1.txt', 'world_war2.txt', 'world_war3.txt', 'world_war4.txt', 'world_war5.txt', 'einstein1.txt', 'einstein2.txt', 'einstein3.txt', 'einstein4.txt', 'einstein5.txt']
Cluster 1: ['velvet_revolution1.txt', 'velvet_revolution2.txt', 'velvet_revolution3.txt', 'velvet_revolution4.txt', 'velvet_revolution5.txt', 'charles_bridge1.txt', 'charles_bridge2.txt', 'charles_bridge3.txt', 'charles_bridge4.txt', 'charles_bridge5.txt', 'havel1.txt', 'havel2.txt', 'havel3.txt', 'havel4.txt', 'havel5.txt', 'klaus1.txt', 'klaus2.txt', 'klaus3.txt', 'klaus4.txt', 'klaus5.txt', 'mandela1.txt', 'mandela2.txt', 'mandela3.txt', 'mandela4.txt', 'mandela5.txt', 'gold_havel.txt']
Cluster 2: ['airplane1.txt', 'airplane2.txt', 'airplane3.txt', 'airplane4.txt', 'airplane5.txt', 'boat1.txt', 'boat2.txt', 'boat3.txt', 'boat4.txt', 'boat5.txt', 'car1.txt', 'car2.txt', 'car3.txt', 'car4.txt', 'car5.txt', 'train1.txt', 'train2.txt', 'train3.txt', 'train4.txt', 'train5.txt', 'gold_airplane.txt']
Clust

In [9]:
from sklearn.metrics import rand_score
rand_index = rand_score(groundtruth10_gold, clusters10)

print(f"Rand Index: {rand_index}")

Rand Index: 0.9616769195716565


In [10]:
from sklearn.metrics import adjusted_rand_score

ari10 = adjusted_rand_score(groundtruth10_gold, clusters10)
print(f"Adjusted Rand Index: {ari10:.4f}")

Adjusted Rand Index: 0.7898


In [11]:
from sklearn.metrics import pair_confusion_matrix
def pairwise_prf(y_true, y_pred):
    tn, fp, fn, tp = pair_confusion_matrix(y_true, y_pred).ravel()
    print("fp:", fp, "fn:", fn)
    prec = tp / (tp + fp) if tp + fp > 0 else 0.0
    rec  = tp / (tp + fn) if tp + fn > 0 else 0.0
    f1   = 2 * prec * rec / (prec + rec) if prec + rec > 0 else 0.0
    return {"precision": prec, "recall": rec, "f1": f1}

In [12]:
metrics10 = pairwise_prf(groundtruth10_gold, clusters10)
print(f"Pairwise Precision: {metrics10['precision']:.4f}")
print(f"Pairwise Recall:    {metrics10['recall']   :.4f}")
print(f"Pairwise F1-score:  {metrics10['f1']       :.4f}")

fp: 1092 fn: 590
Pairwise Precision: 0.7678
Pairwise Recall:    0.8595
Pairwise F1-score:  0.8111


In [13]:
metrics40 = pairwise_prf(groundtruth40_gold, clusters40)
print(f"Pairwise Precision: {metrics40['precision']:.4f}")
print(f"Pairwise Recall:    {metrics40['recall']   :.4f}")
print(f"Pairwise F1-score:  {metrics40['f1']       :.4f}")

fp: 60 fn: 10
Pairwise Precision: 0.9368
Pairwise Recall:    0.9889
Pairwise F1-score:  0.9622
